# GTFS Bus Data Analysis

This notebook analyzes NYC MTA bus GTFS data including:
1. **GTFS Schedule Data** - Static transit schedule from `data/gtfs_b/`
2. **GTFS Realtime Data** - Real-time vehicle positions from parquet files

## Part 1: GTFS Schedule Summary

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load GTFS schedule files
gtfs_path = 'data/gtfs_b/'

print("Loading GTFS Schedule Data...")
print("="*60)

# Load key GTFS files
calendar = pd.read_csv(gtfs_path + 'calendar.txt')
calendar_dates = pd.read_csv(gtfs_path + 'calendar_dates.txt')
routes = pd.read_csv(gtfs_path + 'routes.txt')
trips = pd.read_csv(gtfs_path + 'trips.txt')
stops = pd.read_csv(gtfs_path + 'stops.txt')
agency = pd.read_csv(gtfs_path + 'agency.txt')

print("✓ Loaded all GTFS files successfully")

Loading GTFS Schedule Data...
✓ Loaded all GTFS files successfully


In [2]:
# Service Date Coverage
print("\n📅 SERVICE DATE COVERAGE")
print("="*60)

# Convert date columns to datetime
calendar['start_date'] = pd.to_datetime(calendar['start_date'], format='%Y%m%d')
calendar['end_date'] = pd.to_datetime(calendar['end_date'], format='%Y%m%d')
calendar_dates['date'] = pd.to_datetime(calendar_dates['date'], format='%Y%m%d')

# Calendar (regular service patterns)
print("\nRegular Service Patterns (calendar.txt):")
print(f"  Service start: {calendar['start_date'].min().strftime('%Y-%m-%d')}")
print(f"  Service end:   {calendar['end_date'].max().strftime('%Y-%m-%d')}")
print(f"  Number of service patterns: {len(calendar)}")
print(f"\n  Service IDs by day of week:")
for idx, row in calendar.iterrows():
    days = []
    if row['monday'] == 1: days.append('Mon')
    if row['tuesday'] == 1: days.append('Tue')
    if row['wednesday'] == 1: days.append('Wed')
    if row['thursday'] == 1: days.append('Thu')
    if row['friday'] == 1: days.append('Fri')
    if row['saturday'] == 1: days.append('Sat')
    if row['sunday'] == 1: days.append('Sun')
    print(f"    {row['service_id']}: {', '.join(days)}")

# Calendar dates (exceptions)
print(f"\nService Exceptions (calendar_dates.txt):")
print(f"  Date range: {calendar_dates['date'].min().strftime('%Y-%m-%d')} to {calendar_dates['date'].max().strftime('%Y-%m-%d')}")
print(f"  Total exception dates: {len(calendar_dates)}")
print(f"  Added service (1): {(calendar_dates['exception_type'] == 1).sum()} dates")
print(f"  Removed service (2): {(calendar_dates['exception_type'] == 2).sum()} dates")

# Overall date coverage
all_dates = pd.concat([
    calendar[['start_date']].rename(columns={'start_date': 'date'}),
    calendar[['end_date']].rename(columns={'end_date': 'date'}),
    calendar_dates[['date']]
])
print(f"\n📊 OVERALL DATE RANGE: {all_dates['date'].min().strftime('%Y-%m-%d')} to {all_dates['date'].max().strftime('%Y-%m-%d')}")


📅 SERVICE DATE COVERAGE

Regular Service Patterns (calendar.txt):
  Service start: 2025-01-29
  Service end:   2025-06-28
  Number of service patterns: 29

  Service IDs by day of week:
    EN_B5-Sunday: Sun
    EN_B5-Weekday: Mon, Tue, Wed, Thu, Fri
    EN_B5-Weekday-SDon: Mon, Tue, Wed, Thu, Fri
    EN_B5-Saturday: Sat
    FB_B5-Sunday: Sun
    FB_B5-Weekday: Mon, Tue, Wed, Thu, Fri
    FB_B5-Weekday-SDon: Mon, Tue, Wed, Thu, Fri
    FB_B5-Saturday: Sat
    FP_B5-Sunday: Sun
    FP_B5-Weekday: Mon, Tue, Wed, Thu, Fri
    FP_B5-Weekday-SDon: Mon, Tue, Wed, Thu, Fri
    FP_B5-Saturday: Sat
    GA_B5-Sunday: Sun
    GA_B5-Weekday: Mon, Tue, Wed, Thu, Fri
    GA_B5-Weekday-SDon: Mon, Tue, Wed, Thu, Fri
    GA_B5-Saturday: Sat
    GA_B5-Saturday-BM: Fri
    JG_B5-Sunday: Sun
    JG_B5-Sunday-BM: Sat
    JG_B5-Weekday: Mon, Tue, Wed, Thu, Fri
    JG_B5-Weekday-BM: Mon, Tue, Wed, Thu, Sun
    JG_B5-Weekday-SDon: Mon, Tue, Wed, Thu, Fri
    JG_B5-Weekday-SDon-BM: Mon, Tue, Wed, Thu, Sun
   

In [3]:
# Routes Summary
print("\n🚌 ROUTES SUMMARY")
print("="*60)

print(f"\nTotal routes: {len(routes)}")
print(f"Route types:")
route_types = routes['route_type'].value_counts()
route_type_names = {3: 'Bus', 0: 'Light Rail', 1: 'Subway/Metro', 2: 'Rail'}
for route_type, count in route_types.items():
    type_name = route_type_names.get(route_type, f'Type {route_type}')
    print(f"  {type_name}: {count} routes")

print(f"\nSample routes:")
print(routes[['route_id', 'route_short_name', 'route_long_name', 'route_type']].head(10).to_string(index=False))


🚌 ROUTES SUMMARY

Total routes: 314
Route types:
  Bus: 274 routes
  Type 711: 40 routes

Sample routes:
route_id route_short_name                  route_long_name  route_type
      B1               B1      Bay Ridge - Manhattan Beach           3
     B11              B11            Sunset Park - Midwood           3
     B12              B12 Lefferts Gardens - East New York           3
     B13              B13  Spring Creek - Wyckoff Hospital           3
     B14              B14     Spring Creek - Crown Heights           3
     B15              B15 Bedford Stuyvesant - JFK Airport           3
     B16              B16     Bay Ridge - Lefferts Gardens           3
     B17              B17         Canarsie - Crown Heights           3
      B2               B2  Kings Hwy Station - Kings Plaza           3
     B20              B20         Ridgewood - Spring Creek           3


In [4]:
# Trips Summary
print("\n🚏 TRIPS SUMMARY")
print("="*60)

print(f"\nTotal scheduled trips: {len(trips):,}")
print(f"Unique routes with trips: {trips['route_id'].nunique()}")
print(f"Unique service IDs: {trips['service_id'].nunique()}")

# Trips per route
trips_per_route = trips.groupby('route_id').size().sort_values(ascending=False)
print(f"\nTop 10 routes by number of scheduled trips:")
for route_id, count in trips_per_route.head(10).items():
    route_name = routes[routes['route_id'] == route_id]['route_short_name'].values[0] if len(routes[routes['route_id'] == route_id]) > 0 else route_id
    print(f"  {route_name:6s} ({route_id}): {count:,} trips")

# Direction distribution
if 'direction_id' in trips.columns:
    print(f"\nTrips by direction:")
    print(f"  Direction 0: {(trips['direction_id'] == 0).sum():,}")
    print(f"  Direction 1: {(trips['direction_id'] == 1).sum():,}")


🚏 TRIPS SUMMARY

Total scheduled trips: 45,866
Unique routes with trips: 65
Unique service IDs: 29

Top 10 routes by number of scheduled trips:
  B6     (B6): 1,806 trips
  B41    (B41): 1,737 trips
  Q58    (Q58): 1,688 trips
  B35    (B35): 1,450 trips
  B38    (B38): 1,220 trips
  B46    (B46): 1,188 trips
  B15    (B15): 1,165 trips
  B46-SBS (B46+): 1,117 trips
  B17    (B17): 1,088 trips
  B12    (B12): 1,042 trips

Trips by direction:
  Direction 0: 22,715
  Direction 1: 23,151


In [5]:
# Stops Summary
print("\n📍 STOPS SUMMARY")
print("="*60)

print(f"\nTotal stops: {len(stops):,}")

# Geographic bounds
print(f"\nGeographic coverage:")
print(f"  Latitude range:  {stops['stop_lat'].min():.6f} to {stops['stop_lat'].max():.6f}")
print(f"  Longitude range: {stops['stop_lon'].min():.6f} to {stops['stop_lon'].max():.6f}")

# Location types if available
if 'location_type' in stops.columns:
    print(f"\nLocation types:")
    location_types = stops['location_type'].value_counts()
    location_type_names = {0: 'Stop/Platform', 1: 'Station', 2: 'Entrance/Exit', 3: 'Generic Node', 4: 'Boarding Area'}
    for loc_type, count in location_types.items():
        type_name = location_type_names.get(loc_type, f'Type {loc_type}')
        print(f"  {type_name}: {count:,}")

print(f"\nSample stops:")
print(stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].head(5).to_string(index=False))


📍 STOPS SUMMARY

Total stops: 4,619

Geographic coverage:
  Latitude range:  40.572635 to 40.762524
  Longitude range: -74.040876 to -73.789580

Location types:
  Stop/Platform: 4,619

Sample stops:
 stop_id                  stop_name  stop_lat   stop_lon
  300000 ORIENTAL BLVD/MACKENZIE ST 40.578350 -73.940029
  300002   ORIENTAL BLVD/JAFFRAY ST 40.578066 -73.943029
  300003  ORIENTAL BLVD/HASTINGS ST 40.577909 -73.944643
  300004  ORIENTAL BLVD/FALMOUTH ST 40.577718 -73.946200
  300006     ORIENTAL BLVD/DOVER ST 40.577353 -73.949552


In [6]:
# Overall GTFS Schedule Summary
print("\n" + "="*60)
print("📊 GTFS SCHEDULE SUMMARY")
print("="*60)

print(f"\nAgency: {agency['agency_name'].values[0] if len(agency) > 0 else 'N/A'}")
print(f"\nService Period: {calendar['start_date'].min().strftime('%Y-%m-%d')} to {calendar['end_date'].max().strftime('%Y-%m-%d')}")
print(f"Total Routes: {len(routes):,}")
print(f"Total Trips: {len(trips):,}")
print(f"Total Stops: {len(stops):,}")
print(f"Average Trips per Route: {len(trips) / len(routes):.1f}")

print("\n" + "="*60)
print("Ready to analyze realtime data!")


📊 GTFS SCHEDULE SUMMARY

Agency: MTA New York City Transit

Service Period: 2025-01-29 to 2025-06-28
Total Routes: 314
Total Trips: 45,866
Total Stops: 4,619
Average Trips per Route: 146.1

Ready to analyze realtime data!


---
## Part 2: GTFS Realtime Data Analysis

In [7]:
# Load realtime vehicle position data
print("Loading Realtime Data...")
print("="*60)

df = pd.read_parquet('data/COMPACTED_nyct_mta_bus_gtfsrt_2025-06-02.parquet')

print(f"✓ Loaded {len(df):,} realtime vehicle position records")
print(f"  Date: June 2, 2025")
print(f"  Time range: {df['vehicle.timestamp'].min()} to {df['vehicle.timestamp'].max()}")
print(f"  Total routes: {df['vehicle.trip.route_id'].nunique()}")
print(f"  Total trips: {df['vehicle.trip.trip_id'].nunique()}")

Loading Realtime Data...
✓ Loaded 2,365,540 realtime vehicle position records
  Date: June 2, 2025
  Time range: 2025-06-01 18:00:08+00:00 to 2025-06-02 18:01:02+00:00
  Total routes: 334
  Total trips: 46133


In [8]:
# Filter for B46 route
print("\n🚌 Filtering for B46 Route")
print("="*60)

df_b46 = df[df['vehicle.trip.route_id'] == 'B46'].copy()

print(f"\nB46 Route Statistics:")
print(f"  Total records: {len(df_b46):,}")
print(f"  Unique trips: {df_b46['vehicle.trip.trip_id'].nunique()}")
print(f"  Unique vehicles: {df_b46['vehicle.vehicle.id'].nunique()}")
print(f"  Time range: {df_b46['vehicle.timestamp'].min()} to {df_b46['vehicle.timestamp'].max()}")

# Check for records without trip_id
missing_trip_id = df_b46['vehicle.trip.trip_id'].isna().sum()
if missing_trip_id > 0:
    print(f"  ⚠️  Records without trip_id: {missing_trip_id}")
    df_b46 = df_b46.dropna(subset=['vehicle.trip.trip_id'])
    print(f"  Continuing with {len(df_b46):,} records that have trip_ids")

df_b46.head()


🚌 Filtering for B46 Route

B46 Route Statistics:
  Total records: 18,181
  Unique trips: 285
  Unique vehicles: 67
  Time range: 2025-06-01 18:01:21+00:00 to 2025-06-02 18:00:58+00:00


,id,vehicle.trip.trip_id,vehicle.trip.start_date,vehicle.trip.route_id,vehicle.trip.direction_id,vehicle.position.latitude,vehicle.position.longitude,vehicle.position.bearing,vehicle.timestamp,vehicle.stop_id,vehicle.vehicle.id,vehicle.occupancy_status
149,MTA NYCT_7944,FB_B5-Sunday-076500_B46_426,20250601,B46,0.0,40.692123,-73.930901,101.309929,2025-06-01 18:01:51+00:00,303630,MTA NYCT_7944,1.0
156,MTA NYCT_7967,FB_B5-Sunday-078200_B46_424,20250601,B46,0.0,40.635677,-73.928665,96.277298,2025-06-01 18:01:51+00:00,303601,MTA NYCT_7967,1.0
165,MTA NYCT_7956,FB_B5-Sunday-082500_B46_428,20250601,B46,0.0,40.639534,-73.929077,96.097580,2025-06-01 18:01:52+00:00,303602,MTA NYCT_7956,1.0
546,MTA NYCT_4621,FB_B5-Sunday-083300_B46_417,20250601,B46,0.0,40.629070,-73.927956,96.324562,2025-06-01 18:01:49+00:00,303597,MTA NYCT_4621,NaN
548,MTA NYCT_4628,FB_B5-Sunday-080000_B46_414,20250601,B46,1.0,40.673115,-73.930710,264.891113,2025-06-01 18:01:37+00:00,303676,MTA NYCT_4628,NaN


In [9]:
# Create trip-level summary with observation counts
print("\n📊 B46 Trip-Level Summary")
print("="*60)

# Count observations per trip
b46_trip_summary = df_b46.groupby('vehicle.trip.trip_id').agg({
    'vehicle.trip.trip_id': 'count',
    'vehicle.trip.direction_id': 'first',
    'vehicle.timestamp': ['min', 'max']
}).reset_index()

# Flatten column names
b46_trip_summary.columns = ['trip_id', 'num_observations', 'direction_id', 'first_timestamp', 'last_timestamp']

# Calculate duration
b46_trip_summary['duration_minutes'] = (
    b46_trip_summary['last_timestamp'] - b46_trip_summary['first_timestamp']
).dt.total_seconds() / 60

print(f"\nCreated summary for {len(b46_trip_summary)} unique B46 trips")
print(f"Observation count range: {b46_trip_summary['num_observations'].min()} to {b46_trip_summary['num_observations'].max()}")
print(f"\nSample:")
b46_trip_summary.head(10)


📊 B46 Trip-Level Summary

Created summary for 285 unique B46 trips
Observation count range: 1 to 129

Sample:


,trip_id,num_observations,direction_id,first_timestamp,last_timestamp,duration_minutes
0,FB_B5-Sunday-074800_B46_413,15,0.0,2025-06-01 18:01:39+00:00,2025-06-01 18:15:04+00:00,13.416667
1,FB_B5-Sunday-076500_B46_426,2,0.0,2025-06-01 18:01:51+00:00,2025-06-01 18:02:22+00:00,0.516667
2,FB_B5-Sunday-076600_B46_418,10,1.0,2025-06-01 18:01:21+00:00,2025-06-01 18:11:48+00:00,10.450000
3,FB_B5-Sunday-077400_B46_423,12,0.0,2025-06-01 18:01:30+00:00,2025-06-01 18:14:19+00:00,12.816667
4,FB_B5-Sunday-078200_B46_424,24,0.0,2025-06-01 18:01:51+00:00,2025-06-01 18:24:21+00:00,22.500000
5,FB_B5-Sunday-078300_B46_412,73,1.0,2025-06-01 18:01:48+00:00,2025-06-01 19:12:55+00:00,71.116667
6,FB_B5-Sunday-079100_B46_410,77,1.0,2025-06-01 18:01:49+00:00,2025-06-01 19:20:05+00:00,78.266667
7,FB_B5-Sunday-079100_B46_415,30,0.0,2025-06-01 18:01:34+00:00,2025-06-01 18:30:29+00:00,28.916667
8,FB_B5-Sunday-079900_B46_427,45,0.0,2025-06-01 18:01:24+00:00,2025-06-01 18:47:44+00:00,46.333333
9,FB_B5-Sunday-080000_B46_414,67,1.0,2025-06-01 18:01:37+00:00,2025-06-01 19:22:21+00:00,80.733333


In [10]:
# Check if trips exist in GTFS schedule
print("\n🔍 Checking Trips Against GTFS Schedule")
print("="*60)

# Filter GTFS trips for B46 route
gtfs_b46_trips = trips[trips['route_id'] == 'B46'].copy()

print(f"\nGTFS Schedule for B46:")
print(f"  Scheduled trips in GTFS: {len(gtfs_b46_trips)}")
print(f"  Observed trips in realtime: {len(b46_trip_summary)}")

# Check which observed trips are in the schedule
b46_trip_summary['in_schedule'] = b46_trip_summary['trip_id'].isin(gtfs_b46_trips['trip_id'])

in_schedule_count = b46_trip_summary['in_schedule'].sum()
not_in_schedule_count = (~b46_trip_summary['in_schedule']).sum()

print(f"\nMatch Results:")
print(f"  ✓ Trips found in schedule: {in_schedule_count}")
print(f"  ✗ Trips NOT in schedule: {not_in_schedule_count}")
print(f"  Match rate: {in_schedule_count / len(b46_trip_summary) * 100:.1f}%")

# Show some examples of trips not in schedule
if not_in_schedule_count > 0:
    print(f"\nSample trips NOT found in schedule:")
    print(b46_trip_summary[~b46_trip_summary['in_schedule']][['trip_id', 'num_observations', 'direction_id']].head())


🔍 Checking Trips Against GTFS Schedule

GTFS Schedule for B46:
  Scheduled trips in GTFS: 1188
  Observed trips in realtime: 285

Match Results:
  ✓ Trips found in schedule: 285
  ✗ Trips NOT in schedule: 0
  Match rate: 100.0%


In [11]:
# Get first stop information from schedule
print("\n🚏 Getting First Stop Information from Schedule")
print("="*60)

print("Loading stop_times.txt (this may take a moment)...")
stop_times = pd.read_csv(gtfs_path + 'stop_times.txt')

print(f"✓ Loaded {len(stop_times):,} stop time records")

# Get first stop for each B46 trip (stop_sequence = 1 or minimum)
b46_trip_ids = gtfs_b46_trips['trip_id'].tolist()
stop_times_b46 = stop_times[stop_times['trip_id'].isin(b46_trip_ids)].copy()

print(f"  Stop times for B46 trips: {len(stop_times_b46):,}")

# Get the first stop for each trip (lowest stop_sequence)
first_stops = stop_times_b46.loc[stop_times_b46.groupby('trip_id')['stop_sequence'].idxmin()][
    ['trip_id', 'arrival_time', 'stop_id', 'stop_sequence']
].copy()

print(f"  First stops identified: {len(first_stops)}")

# Join with stops to get stop names
first_stops = first_stops.merge(
    stops[['stop_id', 'stop_name']], 
    on='stop_id', 
    how='left'
)

print(f"\nSample first stops:")
print(first_stops.head())


🚏 Getting First Stop Information from Schedule
Loading stop_times.txt (this may take a moment)...
✓ Loaded 1,994,835 stop time records
  Stop times for B46 trips: 60,471
  First stops identified: 1188

Sample first stops:
                         trip_id arrival_time  stop_id  stop_sequence  \
0  FB_B5-Saturday-007800_B46_401     01:18:00   307920              1   
1  FB_B5-Saturday-013800_B46_402     02:18:00   307920              1   
2  FB_B5-Saturday-015000_B46_401     02:30:00   307919              1   
3  FB_B5-Saturday-019800_B46_403     03:18:00   307920              1   
4  FB_B5-Saturday-021000_B46_402     03:30:00   307919              1   

                          stop_name  
0              KINGS PLAZA/AVENUE U  
1              KINGS PLAZA/AVENUE U  
2  WILLIAMSBURG BRIDGE PLAZA/LANE 4  
3              KINGS PLAZA/AVENUE U  
4  WILLIAMSBURG BRIDGE PLAZA/LANE 4  


In [12]:
# Create final trip analysis table
print("\n📋 Final B46 Trip Analysis Table")
print("="*60)

# Merge trip summary with first stop information
b46_final = b46_trip_summary.merge(
    first_stops[['trip_id', 'arrival_time', 'stop_name', 'stop_id']], 
    on='trip_id', 
    how='left'
)

# Rename columns for clarity
b46_final = b46_final.rename(columns={
    'arrival_time': 'scheduled_first_stop_time',
    'stop_name': 'scheduled_first_stop_name',
    'stop_id': 'scheduled_first_stop_id'
})

# Sort by in_schedule (True first) and then by num_observations (descending)
b46_final = b46_final.sort_values(['in_schedule', 'num_observations'], ascending=[False, False])

print(f"\nFinal table created with {len(b46_final)} trips")
print(f"\nColumns:")
for col in b46_final.columns:
    print(f"  - {col}")

print(f"\n{'='*60}")
print("Summary Statistics:")
print(f"  Trips in schedule: {b46_final['in_schedule'].sum()}")
print(f"  Trips not in schedule: {(~b46_final['in_schedule']).sum()}")
print(f"  Average observations per trip: {b46_final['num_observations'].mean():.1f}")
print(f"  Median observations per trip: {b46_final['num_observations'].median():.1f}")

# Display the final table
b46_final


📋 Final B46 Trip Analysis Table

Final table created with 285 trips

Columns:
  - trip_id
  - num_observations
  - direction_id
  - first_timestamp
  - last_timestamp
  - duration_minutes
  - in_schedule
  - scheduled_first_stop_time
  - scheduled_first_stop_name
  - scheduled_first_stop_id

Summary Statistics:
  Trips in schedule: 285
  Trips not in schedule: 0
  Average observations per trip: 63.8
  Median observations per trip: 69.0


,trip_id,num_observations,direction_id,first_timestamp,last_timestamp,duration_minutes,in_schedule,scheduled_first_stop_time,scheduled_first_stop_name,scheduled_first_stop_id
132,FB_B5-Weekday-SDon-021000_B46_402,129,1.0,2025-06-02 07:16:36+00:00,2025-06-02 08:40:35+00:00,83.983333,True,03:30:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,307919
97,FB_B5-Sunday-127600_B46_429,127,1.0,2025-06-02 01:08:07+00:00,2025-06-02 03:31:37+00:00,143.500000,True,21:16:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,307919
160,FB_B5-Weekday-SDon-039200_B46_415,125,0.0,2025-06-02 10:35:30+00:00,2025-06-02 12:02:43+00:00,87.216667,True,06:32:00,KINGS PLAZA/AVENUE U,307920
238,FB_B5-Weekday-SDon-062700_B46_407,113,0.0,2025-06-02 14:33:38+00:00,2025-06-02 15:51:11+00:00,77.550000,True,10:27:00,KINGS PLAZA/AVENUE U,307920
56,FB_B5-Sunday-103600_B46_431,109,0.0,2025-06-01 21:36:35+00:00,2025-06-01 23:24:35+00:00,108.000000,True,17:16:00,KINGS PLAZA/AVENUE U,307920
...,...,...,...,...,...,...,...,...,...,...
39,FB_B5-Sunday-095200_B46_434,2,0.0,2025-06-01 19:56:45+00:00,2025-06-01 19:57:15+00:00,0.500000,True,15:52:00,KINGS PLAZA/AVENUE U,307920
283,FB_B5-Weekday-SDon-083700_B46_409,2,1.0,2025-06-02 17:59:26+00:00,2025-06-02 18:00:58+00:00,1.533333,True,13:57:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,307919
98,FB_B5-Sunday-128400_B46_440,1,0.0,2025-06-02 02:47:24+00:00,2025-06-02 02:47:24+00:00,0.000000,True,21:24:00,KINGS PLAZA/AVENUE U,307920
112,FB_B5-Sunday-138400_B46_434,1,1.0,2025-06-02 03:58:27+00:00,2025-06-02 03:58:27+00:00,0.000000,True,23:04:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,307919


In [13]:
# Additional insights and filtered views
print("\n🔎 Additional Insights")
print("="*60)

# Show trips in schedule vs not
print("\n1. TRIPS IN SCHEDULE:")
print("-" * 60)
in_sched = b46_final[b46_final['in_schedule']].copy()
print(f"   Count: {len(in_sched)}")
if len(in_sched) > 0:
    print(f"   Observations: {in_sched['num_observations'].sum():,} total")
    print(f"   Average duration: {in_sched['duration_minutes'].mean():.1f} minutes")
    print(f"\n   Sample (top 10 by observations):")
    display(in_sched[['trip_id', 'num_observations', 'scheduled_first_stop_time', 
                       'scheduled_first_stop_name', 'duration_minutes']].head(10))

print("\n2. TRIPS NOT IN SCHEDULE:")
print("-" * 60)
not_in_sched = b46_final[~b46_final['in_schedule']].copy()
print(f"   Count: {len(not_in_sched)}")
if len(not_in_sched) > 0:
    print(f"   Observations: {not_in_sched['num_observations'].sum():,} total")
    print(f"   Average duration: {not_in_sched['duration_minutes'].mean():.1f} minutes")
    print(f"\n   Sample (top 10 by observations):")
    display(not_in_sched[['trip_id', 'num_observations', 'direction_id', 
                           'first_timestamp', 'duration_minutes']].head(10))

# Direction breakdown
print("\n3. BREAKDOWN BY DIRECTION:")
print("-" * 60)
direction_stats = b46_final.groupby('direction_id').agg({
    'trip_id': 'count',
    'num_observations': 'sum',
    'in_schedule': 'sum'
}).rename(columns={'trip_id': 'trip_count', 'in_schedule': 'scheduled_trip_count'})
print(direction_stats)


🔎 Additional Insights

1. TRIPS IN SCHEDULE:
------------------------------------------------------------
   Count: 285
   Observations: 18,181 total
   Average duration: 65.8 minutes

   Sample (top 10 by observations):


,trip_id,num_observations,scheduled_first_stop_time,scheduled_first_stop_name,duration_minutes
132,FB_B5-Weekday-SDon-021000_B46_402,129,03:30:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,83.983333
97,FB_B5-Sunday-127600_B46_429,127,21:16:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,143.500000
160,FB_B5-Weekday-SDon-039200_B46_415,125,06:32:00,KINGS PLAZA/AVENUE U,87.216667
238,FB_B5-Weekday-SDon-062700_B46_407,113,10:27:00,KINGS PLAZA/AVENUE U,77.550000
56,FB_B5-Sunday-103600_B46_431,109,17:16:00,KINGS PLAZA/AVENUE U,108.000000
129,FB_B5-Weekday-SDon-013800_B46_402,106,02:18:00,KINGS PLAZA/AVENUE U,73.966667
262,FB_B5-Weekday-SDon-073800_B46_421,105,12:18:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,103.950000
243,FB_B5-Weekday-SDon-064800_B46_415,104,10:48:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,132.216667
78,FB_B5-Sunday-114600_B46_443,103,19:06:00,KINGS PLAZA/AVENUE U,102.000000
24,FB_B5-Sunday-087800_B46_427,102,14:38:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,102.950000



2. TRIPS NOT IN SCHEDULE:
------------------------------------------------------------
   Count: 0

3. BREAKDOWN BY DIRECTION:
------------------------------------------------------------
              trip_count  num_observations  scheduled_trip_count
direction_id                                                    
0.0                  142              9118                   142
1.0                  143              9063                   143


In [14]:
# Create a clean export version of the table
print("\n💾 Export-Ready Table")
print("="*60)

# Select and reorder columns for export
export_columns = [
    'trip_id',
    'num_observations', 
    'in_schedule',
    'scheduled_first_stop_time',
    'scheduled_first_stop_name',
    'direction_id',
    'first_timestamp',
    'last_timestamp',
    'duration_minutes'
]

b46_export = b46_final[export_columns].copy()

# Format the table for better readability
b46_export['in_schedule'] = b46_export['in_schedule'].map({True: 'Yes', False: 'No'})
b46_export['duration_minutes'] = b46_export['duration_minutes'].round(1)

print(f"\nExport table ready with {len(b46_export)} rows and {len(b46_export.columns)} columns")
print(f"\nYou can export this to CSV with:")
print("  b46_export.to_csv('b46_trip_analysis.csv', index=False)")

print(f"\nFull table (showing all {len(b46_export)} rows):")

# Display all rows without truncation
pd.set_option('display.max_rows', None)
b46_export


💾 Export-Ready Table

Export table ready with 285 rows and 9 columns

You can export this to CSV with:
  b46_export.to_csv('b46_trip_analysis.csv', index=False)

Full table (showing all 285 rows):


,trip_id,num_observations,in_schedule,scheduled_first_stop_time,scheduled_first_stop_name,direction_id,first_timestamp,last_timestamp,duration_minutes
132,FB_B5-Weekday-SDon-021000_B46_402,129,Yes,03:30:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,1.0,2025-06-02 07:16:36+00:00,2025-06-02 08:40:35+00:00,84.0
97,FB_B5-Sunday-127600_B46_429,127,Yes,21:16:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,1.0,2025-06-02 01:08:07+00:00,2025-06-02 03:31:37+00:00,143.5
160,FB_B5-Weekday-SDon-039200_B46_415,125,Yes,06:32:00,KINGS PLAZA/AVENUE U,0.0,2025-06-02 10:35:30+00:00,2025-06-02 12:02:43+00:00,87.2
238,FB_B5-Weekday-SDon-062700_B46_407,113,Yes,10:27:00,KINGS PLAZA/AVENUE U,0.0,2025-06-02 14:33:38+00:00,2025-06-02 15:51:11+00:00,77.6
56,FB_B5-Sunday-103600_B46_431,109,Yes,17:16:00,KINGS PLAZA/AVENUE U,0.0,2025-06-01 21:36:35+00:00,2025-06-01 23:24:35+00:00,108.0
129,FB_B5-Weekday-SDon-013800_B46_402,106,Yes,02:18:00,KINGS PLAZA/AVENUE U,0.0,2025-06-02 06:01:36+00:00,2025-06-02 07:15:34+00:00,74.0
262,FB_B5-Weekday-SDon-073800_B46_421,105,Yes,12:18:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,1.0,2025-06-02 16:16:45+00:00,2025-06-02 18:00:42+00:00,104.0
243,FB_B5-Weekday-SDon-064800_B46_415,104,Yes,10:48:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,1.0,2025-06-02 14:35:40+00:00,2025-06-02 16:47:53+00:00,132.2
78,FB_B5-Sunday-114600_B46_443,103,Yes,19:06:00,KINGS PLAZA/AVENUE U,0.0,2025-06-01 23:25:37+00:00,2025-06-02 01:07:37+00:00,102.0
24,FB_B5-Sunday-087800_B46_427,102,Yes,14:38:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,1.0,2025-06-01 18:48:46+00:00,2025-06-01 20:31:43+00:00,103.0


In [15]:
# Calculate straight-line distance and average speed for data quality check
print("\n📏 Distance and Speed Analysis")
print("="*60)

from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    Returns distance in miles
    """
    # Convert to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    # Radius of earth in miles
    r = 3959
    
    return r * c

# Get first and last position for each trip
first_positions = df_b46.loc[df_b46.groupby('vehicle.trip.trip_id')['vehicle.timestamp'].idxmin()][
    ['vehicle.trip.trip_id', 'vehicle.position.latitude', 'vehicle.position.longitude']
].rename(columns={
    'vehicle.trip.trip_id': 'trip_id',
    'vehicle.position.latitude': 'first_lat',
    'vehicle.position.longitude': 'first_lon'
})

last_positions = df_b46.loc[df_b46.groupby('vehicle.trip.trip_id')['vehicle.timestamp'].idxmax()][
    ['vehicle.trip.trip_id', 'vehicle.position.latitude', 'vehicle.position.longitude']
].rename(columns={
    'vehicle.trip.trip_id': 'trip_id',
    'vehicle.position.latitude': 'last_lat',
    'vehicle.position.longitude': 'last_lon'
})

# Merge positions into b46_final
b46_with_distance = b46_final.merge(first_positions, on='trip_id', how='left')
b46_with_distance = b46_with_distance.merge(last_positions, on='trip_id', how='left')

# Calculate straight-line distance
b46_with_distance['distance_miles'] = b46_with_distance.apply(
    lambda row: haversine_distance(
        row['first_lat'], row['first_lon'],
        row['last_lat'], row['last_lon']
    ) if pd.notna(row['first_lat']) else 0,
    axis=1
)

# Calculate average speed in mph (distance / duration in hours)
b46_with_distance['duration_hours'] = b46_with_distance['duration_minutes'] / 60
b46_with_distance['avg_speed_mph'] = b46_with_distance.apply(
    lambda row: row['distance_miles'] / row['duration_hours'] if row['duration_hours'] > 0 else 0,
    axis=1
)

# Count unique stops per trip
unique_stops = df_b46.groupby('vehicle.trip.trip_id')['vehicle.stop_id'].nunique().reset_index()
unique_stops.columns = ['trip_id', 'unique_stops']
b46_with_distance = b46_with_distance.merge(unique_stops, on='trip_id', how='left')

print(f"\n✓ Calculated distances and speeds for {len(b46_with_distance)} trips")
print(f"\nDistance range: {b46_with_distance['distance_miles'].min():.2f} to {b46_with_distance['distance_miles'].max():.2f} miles")
print(f"Speed range: {b46_with_distance['avg_speed_mph'].min():.2f} to {b46_with_distance['avg_speed_mph'].max():.2f} mph")
print(f"Unique stops range: {b46_with_distance['unique_stops'].min()} to {b46_with_distance['unique_stops'].max()}")

# Create final diagnostic table sorted by average speed
diagnostic_columns = [
    'trip_id',
    'duration_minutes',
    'unique_stops',
    'distance_miles',
    'avg_speed_mph',
    'num_observations',
    'scheduled_first_stop_time',
    'scheduled_first_stop_name',
    'direction_id'
]

b46_diagnostic = b46_with_distance[diagnostic_columns].copy()
b46_diagnostic['duration_minutes'] = b46_diagnostic['duration_minutes'].round(1)
b46_diagnostic['distance_miles'] = b46_diagnostic['distance_miles'].round(2)
b46_diagnostic['avg_speed_mph'] = b46_diagnostic['avg_speed_mph'].round(2)

# Sort by average speed (lowest first - indicates incomplete trips)
b46_diagnostic = b46_diagnostic.sort_values('avg_speed_mph', ascending=True)

print(f"\n{'='*60}")
print("DIAGNOSTIC TABLE - Sorted by Average Speed")
print("(Lower speeds may indicate incomplete or bad data)")
print("="*60)

# Display all rows
pd.set_option('display.max_rows', None)
b46_diagnostic


📏 Distance and Speed Analysis

✓ Calculated distances and speeds for 285 trips

Distance range: 0.00 to 7.27 miles
Speed range: 0.00 to 16.37 mph
Unique stops range: 1 to 51

DIAGNOSTIC TABLE - Sorted by Average Speed
(Lower speeds may indicate incomplete or bad data)


,trip_id,duration_minutes,unique_stops,distance_miles,avg_speed_mph,num_observations,scheduled_first_stop_time,scheduled_first_stop_name,direction_id
284,FB_B5-Weekday-SDon-053700_B46_403,0.0,1,0.00,0.00,1,08:57:00,KINGS PLAZA/AVENUE U,0.0
283,FB_B5-Sunday-138400_B46_434,0.0,1,0.00,0.00,1,23:04:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,1.0
282,FB_B5-Sunday-128400_B46_440,0.0,1,0.00,0.00,1,21:24:00,KINGS PLAZA/AVENUE U,0.0
278,FB_B5-Weekday-SDon-085400_B46_415,2.0,1,0.00,0.03,3,14:14:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,1.0
270,FB_B5-Weekday-SDon-063700_B46_418,14.0,2,0.02,0.07,13,10:37:00,KINGS PLAZA/AVENUE U,0.0
272,FB_B5-Sunday-076600_B46_418,10.4,2,0.04,0.23,10,12:46:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,1.0
271,FB_B5-Sunday-077400_B46_423,12.8,2,0.05,0.25,12,12:54:00,KINGS PLAZA/AVENUE U,0.0
281,FB_B5-Weekday-SDon-083700_B46_409,1.5,1,0.02,0.74,2,13:57:00,WILLIAMSBURG BRIDGE PLAZA/LANE 4,1.0
266,FB_B5-Sunday-074800_B46_413,13.4,3,0.30,1.36,15,12:28:00,KINGS PLAZA/AVENUE U,0.0
247,FB_B5-Weekday-SDon-041600_B46_421,50.9,16,1.34,1.59,26,06:56:00,UTICA AV/AVENUE N,0.0
